In [ ]:
import json
import re

import sys
sys.path.append('lib')
from cleaning_helpers import remove_null_items_recursive

In [ ]:
with open("../config.json", 'r') as config_file:
    config = json.load(config_file)
    output_directory = ('../' + config["output_directory"]).replace('//', '/')

with open("intermediate_outputs/dependency_mapped_data.json", 'r') as f:
    data = json.load(f)

# build graph using dot notation

In [ ]:
def build_digraph(keep: list[str]):
    global data;
    node_map = {key: index for index, key in enumerate(keep)}
    colors_map = {
        "widget": ["gainsboro", "black"],
        "datasource": ["#BBD686", "black"],
        "function": ["#F2B5D4", "black"],
        "frame": ["gold2", "black"],
    }

    dot_data = "digraph G {\n"\
    "  node [shape=record, style=filled]\n"\
    "  edge [color=black]\n"

    # define nodes
    for name, index in node_map.items():
        fill_color = "yellow"
        text_color = "black"
        if name in data.keys() and "type" in data[name]:
            _type = data[name].get("type")
            _subtype = data[name].get("subtype")
            
            # colors
            [fill_color, text_color] = colors_map.get(_type)
            if any(subtype in _subtype for subtype in ["Button", "Select"]):
                fill_color = "red4"
                text_color = "white"
            elif "Input" in _subtype:
                fill_color = "lightpink"
            elif "Table" in _subtype:
                fill_color = "darkgreen"
                text_color = "white"
            elif any(subtype in _subtype for subtype in ["Container", "Icon", "Text"]):
                fill_color = "white"

            if _type == "widget":
                if len(data[name]["component_relations"].get("dependants", [])) == 0:
                    fill_color = "black"
                    text_color = "white"

            name = f"{{{name}|{{{_type}|{_subtype}}}}}"
        dot_data += f'  {index} [label="{name}" fillcolor="{fill_color}" fontcolor="{text_color}"];\n'
    dot_data += "\n"


    # helper function
    def get_method_styles(method):
        method_map = {
            "update": {
                "label": "U",
                "color": "deepskyblue",
                "penwidth": "1.5"
            },
            "trigger": {
                "label": "T",
                "color": "red",
                "penwidth": "3"
            },
        }
        color = "gray50"
        penwidth = "1"
        if method in method_map:
            color = method_map.get(method)["color"]
            penwidth = method_map.get(method)["penwidth"]
        return f'['\
            f' color="{color}"'\
            f' penwidth="{penwidth}"'\
            f']'


    links = set([
        f'{node_map.get(d["triggered_by"])} -> {node_map.get(d["pluginId"])} {get_method_styles(d["method"])}'
        for value 
        in data.values() 
            for d 
            in (value["component_relations"].get("dependants") or [])
            if d["pluginId"] in node_map and d["triggered_by"] in node_map
                # and not (
                #     data.get(d["pluginId"], {}).get("type") == "widget"
                #     and len([
                #         dependant
                #         for dependant 
                #         in data.get(d["pluginId"], {}).get("component_relations", {}).get("dependants", [])
                #         if dependant["pluginId"] in node_map    
                #     ]) == 0
                # )
    ])
    dot_data += "\n".join(links)
    dot_data += "\n}"

    return dot_data

In [ ]:
def num_dependants(name: str):
    global data
    visited = set()
    def helper(_name: str):
        if _name in visited:
            return 1
        visited.add(_name)
        return len([
            dependant
            for dependant 
            in data.get(_name, {}).get("component_relations", {}).get("dependants", [])
            if not (
                dependant["type"] == "widget" 
                and helper(dependant["pluginId"]) == 0
            )
        ])
    return helper(name)

def inconsequential_widget(name):
    return False;
    global data
    return data.get(name, {}).get("type") == "widget" and num_dependants(name) == 0

In [ ]:
keep_a = [ 
    key 
    for key
    in data.keys() 
    if not inconsequential_widget(key)
]

keep_b = [ 
    id
    for value 
    in data.values() 
        for id
        in [
            dependant["pluginId"]
            for dependant
            in value["component_relations"].get("dependants", [])
            if not inconsequential_widget(dependant["pluginId"])
        ]
]

In [ ]:
def build_filename(dir_name: str, file_name: str):
    return f"{dir_name}/{file_name}".replace('//', '/')

In [ ]:
keep_all = [
    name 
    for name 
    in list(set(list(keep_a + keep_b))) 
    if not not name
    and name in data
    and data[name].get("type") != "setting"
]

output_file_name = build_filename(output_directory, "full_dependency_graph.dot")
print(output_file_name)
with open(output_file_name, 'w') as f:
    f.write(build_digraph(keep_all))

In [ ]:
keep_queries = [
    name 
    for name 
    in set(keep_a + keep_b) 
    if not not name
    and name in data
    and data[name].get("type") != "setting"
    and data[name].get("type") != "widget"
]

output_file_name = build_filename(output_directory, "queries_only_dependency_graph.dot")
print(output_file_name)
with open(output_file_name, 'w') as f:
    f.write(build_digraph(keep_queries))

In [ ]:
keep_queries_except_employee = [
    name
    for name
    in keep_queries
    if name != 'employee'
]

output_file_name = build_filename(output_directory, "minus_employee.dot")
print(output_file_name)
with open(output_file_name, 'w') as f:
    f.write(build_digraph(keep_queries_except_employee))